In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import string
import pandas as pd
#from amortization.schedule import amortization_schedule
from tabulate import tabulate


# Compound Interest,Simple Interest and EMI Calculator

In [ ]:
def CompoundInterest(P,T,R,Tax,n):
    #Here P is principal
    #T is total time period(in years)
    #R is rate of interest
    #n is the number of times amount is compounded in one year
    
    CI=P*pow((1+(R/(100*n))*(1-(Tax/100))),n*T)-P
    return CI

In [ ]:
def SimpleInterest(P,T,R,Tax,n):
    
    #Here n is number of times Simple Interest is calciulated each year
    SI=P*(n*T)*R/100#Total Interest in that year
    return SI

In [ ]:
def EMICalculator(P,T,R,n):
    EMI=(P*(R/(n*100))*pow(1+R/(n*100),T*n))/(pow(1+R/(n*100),T*n)-1)
    return EMI

# Web Scraping

In [ ]:
CLPage=requests.get("https://www.myloancare.in/car-loan-calculator/#terms")

CLsoup=bs(CLPage.content,'lxml')#Creating soup object

attr={'class':'table table-curved'}
table=CLsoup.find('table',attrs=attr)#Searching for first table with specified attributes containing required data

In [ ]:
table_rows=table.find_all('tr')#Extracting all table rows
Bank_list=np.empty(0)#Contains list of bank names
Interest_Rates=np.empty(0)#Contains list of corresponding interest rates

for tr in table_rows:
    td=tr.find_all('td')
    try:
        row=[tr.text for tr in td]#Extracting row data
        
        Bank_list=np.append(Bank_list,row[0])
        
        temp=row[1].split("%")#Splitting string to get interest rates
        temp2=float(temp[0])
        Interest_Rates=np.append(Interest_Rates,temp2)#Storing it in interest rate array
    except(IndexError):
        continue

In [ ]:
HLPage=requests.get("https://www.myloancare.in/home-loan-interest-rates/")

HLsoup=bs(HLPage.content,'lxml')#Creating soup object

attr2={'class':'table table-curved'}
table2=HLsoup.find('table',attrs=attr2)#Searching for first table with specified attributes containing required data

In [ ]:
table_rows=table.find_all('tr')#Extracting all table rows
Bank_list2=np.empty(0)#Contains list of bank names
Interest_Rates2=np.empty(0)#Contains list of corresponding interest rates

for tr in table_rows:
    td=tr.find_all('td')
    try:
        row=[tr.text for tr in td]#Extracting row data
        
        Bank_list2=np.append(Bank_list2,row[0])
        
        temp=row[1].split("%")#Splitting string to get interest rates
        temp2=float(temp[0])
        Interest_Rates2=np.append(Interest_Rates2,temp2)#Storing it in interest rate array
    except(IndexError):
        continue

# Calculator Functions

In [ ]:
def FDCalc():
    print("Enter Total Investment:")
    i=float(input())
    print("Enter Total Time Period in years:(compunded quarterly)")
    t=int(input())
    print("Enter Rate ofinterest:")
    r=float(input())
    print("Enter Total Tax percent(If any applicable else enter 0):")
    tax=float(input())
    
    n=4#Note this value indicates if we are taking values annually(n=1)
       #semi-annually(n=2), quarterly(n=4) and monthly(n=12) it basically 
       #represents how many times principal is compunded each year
    print((CompoundInterest(i,t,r,tax,n))+i)

In [ ]:
def EMICalc():
    print("Enter Total Investment:")
    i=float(input())
    print("Enter Total Time Period in years:")
    t=int(input())
    print("Enter Rate ofinterest:")
    r=float(input())
    
    n=12#Note this value indicates if we are taking values annually(n=1)
       #semi-annually(n=2), quarterly(n=4) and monthly(n=12) it basically 
       #represents how many times principal is compunded each year   
    sum=EMICalculator(i,t,r,n)
    print(sum)

In [ ]:
def CarEMICalc():
    print("Select your bank:")
    for j in range(len(Bank_list)):
        print((j+1)," : ",Bank_list[j])
    choice=int(input())
    r=Interest_Rates[choice-1]    
    p=float(input("Enter Principal amount"))
    t=int(input("Enter tenure in years:"))
    n=12#Note this value indicates if we are taking values annually(n=1)
       #semi-annually(n=2), quarterly(n=4) and monthly(n=12) it basically 
       #represents how many times principal is compunded each year
    print("Total EMI payable each month:",EMICalculator(p,t,r,n))

In [ ]:
def CarLoanAmortizationSchedule():
    
    #Below we are selecting the bank from which we will take loan and assign the appropriate interest rat
    print("Select your bank:")
    for j in range(len(Bank_list)):
        print((j+1)," : ",Bank_list[j])
    choice=int(input())
    
    r=Interest_Rates[choice-1]    
    
    p=float(input("Enter Principal amount"))
    t=int(input("Enter tenure in years:"))
    n=12#Here we are setting period of monthly duration(n=Number of times EMI is paid per year)
   
    LoanBal=p
    TotalIpaid=0.0#Setting total interest paid so far as zero
    
    #Calculating Amortization Amount below
    Amortization_amt=p*((r/(n*100))*pow(1+r/(n*100),t*n))/(pow(1+r/(n*100),t*n)-1)
    
    CLdf=pd.DataFrame(index=['Total Payment','Principal','Interest','Total Interest','Loan Balance'])
    
    for j in range(0,t*n):
        
        Ipaid=LoanBal*(r/(n*100))#Interest to be paid this period
        TotalIpaid+=Ipaid#Summing up total interest paid
        Ppaid=Amortization_amt-Ipaid#Principal amount to be deducted from Loan bal
        LoanBal=LoanBal-Ppaid#Remaining Loan balance
        
        if LoanBal<0:
            LoanBal=0
            
        data=[MonthlyAmt,Ppaid,Ipaid,TotalIpaid,LoanBal]
        name='Month'+str(j+1)
        
        CLdf[name]=data#Storing data in dataframe
    print(CLdf)

In [ ]:
def HomeEMICalc():
    print("Select your bank:")
    for j in range(len(Bank_list2)):
        print((j+1)," : ",Bank_list2[j])
    choice=int(input())
    
    r=Interest_Rates2[choice-1]    
    p=float(input("Enter Principal amount:"))
    t=int(input("Enter tenure in years:"))
    
    n=12#Note this value indicates if we are taking values annually(n=1)
       #semi-annually(n=2), quarterly(n=4) and monthly(n=12) it basically 
       #represents how many times principal is compunded each year
    print("Total EMI payable each month:",EMICalculator(p,t,r,n))

In [ ]:
def HomeLoanAmortizationSchedule():
    
    #Below we are selecting the bank from which we will take loan and assign the appropriate interest rate
    print("Select your bank:")
    for j in range(len(Bank_list2)):
        print((j+1)," : ",Bank_list2[j])
    choice=int(input())
    
    r=Interest_Rates2[choice-1]    
    p=float(input("Enter Principal amount"))
    t=int(input("Enter tenure in years:"))
    n=12
    
    LoanBal=p
    TotalIpaid=0.0#Setting total interest paid so far as zero
    
    #Calculating Amortization Amount below
    Amortization_amt=p*((r/(n*100))*pow(1+r/(n*100),t*n))/(pow(1+r/(n*100),t*n)-1)
    
    HLdf=pd.DataFrame(index=['Total Payment','Principal','Interest','Total Interest','Loan Balance'])
    
    for j in range(0,t*n):
        
        Ipaid=LoanBal*(r/(n*100))#Interest to be paid this period
        TotalIpaid+=Ipaid#Summing up total interest paid
        Ppaid=Amortization_amt-Ipaid#Principal amount to be deducted from loan balance
        LoanBal=LoanBal-Ppaid#Remaining Loan balance
        
        if LoanBal<0:
            LoanBal=0
            
        data=[MonthlyAmt,Ppaid,Ipaid,TotalIpaid,LoanBal]
        name='Month'+str(j+1)
        
        HLdf[name]=data#Storing output into dataframe
        
    print(HLdf)

In [ ]:
def MFReturns():
    print("Enter Total Investment:")
    i=float(input())
    print("Enter Total Time Period in years:(compunded quarterly)")
    t=int(input())
    print("Enter Rate ofinterest:")
    r=float(input())
    print("Enter Total Tax percent(If any applicable else enter 0):")
    tax=float(input())
    n=1#Note this value indicates if we are taking values annually(n=1)
       #semi-annually(n=2), quarterly(n=4) and monthly(n=12) it basically 
       #represents how many times principal is compunded each year
    print((CompoundInterest(i,t,r,tax,n))+i)

In [ ]:
def RDCalc():
    print("Enter Total Investment:")
    p=float(input())
    print("Enter Total Time Period in years:(compounded quarterly)")
    t=int(input())
    print("Enter Rate ofinterest:")
    r=float(input())
    n=t*4
    Int=(p*(pow((1+r/400),(n))-1))/(1-pow(1+r/400,-1/3))
    print(Int)


# Input 

In [ ]:
#The below dictionary contains the list of functions which conain the calculators
tokenDict = {"1":FDCalc, "2":EMICalc, "3":CarEMICalc,"4":MFReturns,"5":RDCalc,"6":HomeEMICalc,"7":CarLoanAmortizationSchedule,"8":HomeLoanAmortizationSchedule}

#The below dictionary is used to display a menu showing option for each calculator
lines = {'1':'FD Calculator','2':'EMI Calculator','3':'Car Loan EMI Calculator','4':'MF Return Calculator','5':'Recurring Deposit Calculator','6':'Home Loan EMI Calculator','7':'Car Loan Amortization Schedule','8':'Home Loan Amortization Schedule'}
# simulate, say, lines read from a file

for k,v in lines.items():
    print(k," : ",v,"\n")#Displaying Menu

line=str(input())#Taking input

# lookup the function to call that calculator function
functionToCall = tokenDict[line]

# and call it
functionToCall()